In [65]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2


In [70]:
# Load our image

binary_warped = mpimg.imread('warped_example.jpg')



In [71]:
def find_lane_pixels(binary_warped):
    print("enter find_lane_pixels()")
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    print("window_height:")
    print(window_height)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    print("nonzero:")
    print(nonzero)
    print("nonzeroy:")
    print(nonzeroy)
    print("nonzerox:")
    print(nonzerox)
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        
        # Draw the windows on the visualization image        
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        print("good_left_inds:")
        print(good_left_inds)
        print("good_right_inds:")
        print(good_right_inds)
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    print("left_lane_inds:")
    print(left_lane_inds)
    print("right_lane_inds:")
    print(right_lane_inds)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    print("leave find_lane_pixels()")
    return leftx, lefty, rightx, righty, out_img


In [72]:
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    
    print("leftx:")
    print(leftx)
    print("lefty:")
    print(lefty)
    
    print("rightx:")
    print(rightx)
    print("righty:")
    print(righty)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img

In [73]:
out_img = fit_polynomial(binary_warped)
plt.imshow(out_img)


enter find_lane_pixels()
window_height:
80
nonzero:
(array([  0,   0,   0, ..., 719, 719, 719], dtype=int64), array([   0,    0,    0, ..., 1069, 1069, 1069], dtype=int64), array([0, 1, 2, ..., 0, 1, 2], dtype=int64))
nonzeroy:
[  0   0   0 ..., 719 719 719]
nonzerox:
[   0    0    0 ..., 1069 1069 1069]
good_left_inds:
[265956 265957 265958 ..., 291390 291391 291392]
good_right_inds:
[]
good_left_inds:
[238569 238570 238571 ..., 265752 265753 265754]
good_right_inds:
[]
good_left_inds:
[202302 202303 202304 ..., 220674 220675 220676]
good_right_inds:
[]
good_left_inds:
[170877 170878 170879 ..., 202059 202060 202061]
good_right_inds:
[]
good_left_inds:
[137847 137848 137849 ..., 170355 170356 170357]
good_right_inds:
[]
good_left_inds:
[111186 111187 111188 ..., 130902 130903 130904]
good_right_inds:
[]
good_left_inds:
[ 84750  84751  84752 ..., 110940 110941 110942]
good_right_inds:
[]
good_left_inds:
[46002 46003 46004 ..., 84456 84457 84458]
good_right_inds:
[]
good_left_inds:
[  5

TypeError: expected non-empty vector for x